# Cancelación de eco

Supongamos que enviamos una señal de voz a un amig@ 

Nuestro amig@ escucha lo que enviamos en un parlante y responde

Nosotros escuchamos la respuesta de nuestro amig@ y adicionalmente nuestro mensaje original

Podemos eliminar el "eco" con un filtro adaptivo como muestra el siguiente diagrama

<img src="../clases/images/adaptive-echo-canceller.png" width="500">


Se usa como entrada la señal enviada y como salida deseada la señal recibida (con tiene eco)

El objetivo es que el filtro aprenda los coeficientes del sistema reverberante

Finalmente el error entre la salida y la entrada será la señal recibida limpia

Veamos como se programa paso a paso

In [ ]:
import librosa

# Esta será la señal s(t) es decir nuestra voz transmitida
r, Fs = librosa.load("../../data/hola2.ogg")
Audio(r, rate=int(Fs), normalize=False)

In [ ]:
T = 1000
h = np.concatenate((np.zeros(100), 
                    [0.6], np.zeros(T), 
                    [0.4], np.zeros(T), 
                    ))

# Esta es la voz con eco agregado
rh = scipy.signal.convolve(r, h, mode='full')
Audio(rh, rate=int(Fs), normalize=False)

In [ ]:
fig, ax = plt.subplots()
ax.plot(r, alpha=0.5)
ax.plot(rh, alpha=.5)
ax.plot(s, alpha=.5)

En el lado receptor nuestra voz se contamina con eco

<img src="../images/echo-room.png" width="300">

http://dsp-book.narod.ru/307.pdf

In [ ]:
# Esta es la voz de nuestro amig@ denotada por s(t)
s, Fs = librosa.load("../../data/hola1.ogg")
s = np.pad(s, pad_width=(1000, 0))
Audio(s, rate=int(Fs), normalize=False)

In [ ]:
# Esta es la señal de nuestro amigo que nos llega a nosotros
# Es una mezcla de la voz de nuestro amig@ + el eco nuestro + ruido blanco
srh = s + np.pad(rh, pad_width=(0, len(s)-len(rh))) #+ np.random.randn(len(s))*0.005
Audio(srh, rate=int(Fs), normalize=False)

In [ ]:
from numpy.lib.stride_tricks import as_strided


L, mu, delta = 500, 0.1, 1e-10
for L in [100, 200, 500, 1000, 2000, 3000]:
    for mu in [0.01, 0.02, 0.05, 0.1, 0.2, 0.5]:
        hhat = np.zeros(shape=(L+1, ))
        rhhat = np.zeros(shape=(len(srh), ))
        for k in range(L+1, len(r)-1):
            norm = np.sum(r[k-L-1:k]**2) + delta
            rhhat[k] = np.dot(hhat, r[k-L-1:k])
            hhat += 2*mu*(srh[k] - rhhat[k])*r[k-L-1:k]/(norm)
        rhhat[k+1] = np.dot(hhat, r[k-L-1:k])
        # Nuestra estimación de la voz limpia de nuestro amig@
        shat = srh - rhhat
        #shat = shat/np.amax(np.absolute(shat))

        print(L, mu, np.sum((s - shat)**2))

In [ ]:
L, mu, delta = 500, 0.1, 1e-10

hhat = np.zeros(shape=(L+1, ))
rhhat = np.zeros(shape=(len(srh), ))
for k in range(L+1, len(r)-1):
    norm = np.sum(r[k-L-1:k]**2) + delta
    rhhat[k] = np.dot(hhat, r[k-L-1:k])
    hhat += 2*mu*(srh[k] - rhhat[k])*r[k-L-1:k]/(norm)
rhhat[k+1] = np.dot(hhat, r[k-L-1:k])
# Nuestra estimación de la voz limpia de nuestro amig@
shat = srh - rhhat
#shat = shat/np.amax(np.absolute(shat))

fig, ax = plt.subplots(3, figsize=(7, 4), tight_layout=True)
ax[0].plot(r, alpha=0.5, label='r(t)');
ax[0].plot(rh, alpha=0.5, label='r(t)*h(t)');
ax[0].plot(rhhat, alpha=0.5, label='r(t)*hhat(t)');
ax[0].legend()
ax[1].plot(shat, alpha=0.75, label='shat(t)');
ax[1].plot(s, alpha=0.75, label='s(t)');
ax[1].legend()
ax[2].plot((s - shat)**2)

print(np.sum((s - shat)**2))
Audio(shat, rate=int(Fs), normalize=False)

In [ ]:
L, mu, delta = 4000, 0.1, 1e-4

hhat = np.zeros(shape=(L+1, ))
rhhat = np.zeros(shape=(len(srh), ))
for k in range(L+1, len(r)-1):
    norm = np.sum(r[k-L-1:k]**2) + delta
    rhhat[k] = np.dot(hhat, r[k-L-1:k])
    hhat += 2*mu*(rh[k] - rhhat[k])*r[k-L-1:k]/(norm)
rhhat[k+1] = np.dot(hhat, r[k-L-1:k])
# Nuestra estimación de la voz limpia de nuestro amig@
shat = srh - rhhat
#shat = shat/np.amax(np.absolute(shat))

fig, ax = plt.subplots(3, figsize=(7, 4), tight_layout=True)
ax[0].plot(r, alpha=0.5, label='r(t)');
ax[0].plot(rh, alpha=0.5, label='r(t)*h(t)');
ax[0].plot(rhhat, alpha=0.5, label='r(t)*hhat(t)');
ax[0].legend()
ax[1].plot(shat, alpha=0.75, label='shat(t)');
ax[1].plot(s, alpha=0.75, label='s(t)');
ax[1].legend()
ax[2].plot((s - shat)**2)
Audio(shat, rate=int(Fs), normalize=False)